In [1]:
import json
import numpy as np
import copy

In [2]:
def poly_area(x, y, absoluteValue = True):

    result = 0.5 * np.abs(np.dot(x[:-1], y[1:]) + x[-1]*y[0] - np.dot(y[:-1], x[1:]) - y[-1]*x[0])
    if absoluteValue:
        return abs(result)
    else:
        return result

In [3]:
with open("C:\\Users\\dekelmeirom\\OneDrive - Technion\\Documents\\university\\PDL_project\\CCD_Project_28MAR21_all_images.json") as json_file:
    json_data = json.load(json_file)

In [4]:
images = json_data["_via_img_metadata"]

In [5]:
images_names = list(images.keys())
working_images = images_names

In [6]:
for image_name in working_images:
    image = images[image_name]
    for region in image['regions']:
        area = poly_area(region["shape_attributes"]["all_points_x"], region["shape_attributes"]["all_points_y"])
        region["area"] = area

### calculate the total region area for each class in each image

In [7]:
for image_name in working_images:
    positive_area = 0
    negative_area = 0
    image = images[image_name]
    for region in image['regions']:
        if region['region_attributes']['type'] == '2':
            negative_area += region["area"]
        elif region['region_attributes']['type'] == '3':
            positive_area += region["area"]
    image['positive_area'] = positive_area
    image['negative_area'] = negative_area  

In [8]:
print("image_name, positive, negative")
for image_name in working_images:
    image = images[image_name]
    print(image_name + ", " + str(image['positive_area']) + ", " + str(image['negative_area']))

image_name, positive, negative
MOD_1M17 - Annotation 17.png1438654, 12336.5, 259902.0
MOD_1M17 - Annotation 18.png1521050, 0, 109043.0
MOD_1M17 - Annotation 19.png1498537, 24366.0, 58629.5
MOD_1M17 - Annotation 20.png1138511, 0, 0
MOD_1M19_19.06.2019_12.11.28 - Annotation 48.png1566320, 10968.5, 7270.5
MOD_1M19_19.06.2019_12.11.28 - Annotation 49.png1645289, 4794.0, 25034.0
MOD_1M19_19.06.2019_12.11.28 - Annotation 50.png1602084, 42542.0, 90566.5
MOD_1M19_19.06.2019_12.11.28 - Annotation 51.png1641245, 13622.5, 23816.5
MOD_1M19_19.06.2019_12.11.28 - Annotation 53.png1692762, 15971.5, 77411.0
MOD_1M19_19.06.2019_12.11.28 - Annotation 54.png1332395, 2721.0, 41743.5
MOD_1M19_19.06.2019_12.11.28 - Annotation 55.png1714179, 6470.5, 227938.5
MOD_1M19_19.06.2019_12.11.28 - Annotation 56.png1683720, 0, 0
MOD_1M19_19.06.2019_12.11.28 - Annotation 57.png1785011, 68059.0, 442813.5
MOD_1M19 - Annotation 488.png1854943, 129111.5, 392088.0
MOD_1M19 - Annotation 490.png1845456, 105786.5, 6188.0
MOD_1

### split train and test

In [9]:
test_image_names_path = "C:\\Users\\dekelmeirom\\OneDrive - Technion\\Documents\\university\\PDL_project\\test_image_names_all.txt"
with open(test_image_names_path, 'r') as test_image_names_file:
    test_image_names = test_image_names_file.read()
test_image_names = test_image_names.split("\n")
# test_image_names = test_image_names[:-1] # remove the last empty element

In [10]:
# remove bad labeled and duplicated images
ban_list = [test_image_names[5], test_image_names[8], test_image_names[9], test_image_names[-2], test_image_names[-1]]
print(ban_list)

['MOD_1M19 - Annotation 491.png1654528', 'MOD_1M19 - Annotation 500.png1767064', 'MOD_1M19 - Annotation 501.png1783533', '1M05_Default_Extended.png1672532', '1M08_Default_Extended.png1504616']


In [11]:
save_path = "C:\\Users\\dekelmeirom\\OneDrive - Technion\\Documents\\university\\PDL_project\\"
test_path = save_path + "PDL_Project_test_256.json"
train_path = save_path + "PDL_Project_train_256.json"
working_images = images_names #[0:215] #images_names[59:161]
test_images = {}
train_images = {}
for image_name in working_images:
    if image_name in test_image_names:
        if image_name not in ban_list:
            test_images[image_name] = images[image_name]
    else:
        train_images[image_name] = images[image_name]
        
test = copy.deepcopy(json_data)
train = copy.deepcopy(json_data)
test["_via_img_metadata"] = test_images
train["_via_img_metadata"] = train_images

with open(test_path, 'w') as test_file:
    json.dump(test_images, test_file)
with open(train_path, 'w') as train_file:
    json.dump(train_images, train_file)